In [2]:
import fly_analysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter, find_peaks

In [3]:
def filter_trajecs(
    df,
    min_length=300,  # 50 frames = 0.5 seconds
    max_z_speed=0.3,  # shouldn't be flying up/down a lot
    min_speed=0.05,  # should be flying (not wallking)
):
    selected_obj_ids = []

    for obj_id, gdf in df.groupby("obj_id"):
        if len(gdf) > min_length:
            speed = np.linalg.norm(np.vstack([gdf.zvel, gdf.xvel, gdf.yvel]), axis=0)

            if gdf.z.min() > 0.05:  # not on floor
                if gdf.z.max() < 0.5:  # not on ceiling
                    if np.max(np.abs(gdf.zvel)) < max_z_speed:
                        if np.min(speed[5:]) > min_speed:  # first few frames are off
                            selected_obj_ids.append(obj_id)

    return df.loc[df["obj_id"].isin(selected_obj_ids)]

In [4]:
wtcs_files = ["20240916_115831.braidz", "20240917_163455.braidz"]
root_folder = "/home/buchsbaum/mnt/DATA/Experiments/"

In [5]:
wtcs = fa.braidz.read_multiple_braidz(wtcs_files, root_folder)

Reading /home/buchsbaum/mnt/DATA/Experiments/20240916_115831.braidz using pyarrow
Reading /home/buchsbaum/mnt/DATA/Experiments/20240917_163455.braidz using pyarrow


In [9]:
gdf = filter_trajecs(wtcs["df"])
unique_obj_ids = gdf["unique_obj_id"].unique()


32